In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 1.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1623246 sha256=c99fb1de1876dc2597b57efff21f6003949511d0a76f06be2775aff001999142
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

from surprise import KNNWithMeans, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split

## Librerías utilizadas:

**Pandas:** Es una biblioteca de código abierto de Python que proporciona análisis y manipulación de datos en la programación en Python. Es una biblioteca muy prometedora de representación de datos, filtrado y programación estadística. La pieza más importante en pandas es el DataFrame donde almacena y juega con los datos y es lo que usamos para cargar nuesto dataset con el nombre data.csv. (Ebrahim, 2019).

**Numpy:** Es el paquete más básico pero poderoso para la computación científica y la manipulación de datos en Python. Nos permite trabajar con matrices y matrices multidimensionales. (Samaniego, 2021).

**Cosine similarity de Sklearn.metrics.pairwise:** Calcula la similitud como el producto de punto normalizado de X e Y. En los datos normalizados L2, esta función es equivalente a linear_kernel. (PyShark, 2021).

**CSR matrix:** se utiliza para crear una matriz dispersa de formato de fila dispersa compressed, mientras que csc_matrix() se utiliza para crear una matriz dispersa de compressed sparse formato de columna dispersa (GeeksforGeeks, 2020b).

**Svds:** Calcule los valores/vectores singulares k más grandes o más pequeños para una matriz dispersa. El orden de los valores singulares no está garantizado. (ScipyOrg, 2021).

**surprise:** El filtrado colaborativo es una técnica potente que ayuda a que obtengamos mejores resultados de parte de los sistemas de recomendación. Poseen un funcionamiento regido por algoritmos matemáticos que clasifican la información, la estudia y genera sugerencias ajustadas a las necesidades del usuario. (Orlando. P, 2019).

In [ ]:
# Constante
PATH = 'data.csv'

# Import Data

In [ ]:
df = pd.read_csv(PATH)
df.shape

(100000, 10)

In [ ]:
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,655,52,4,11482,300,4,8,2012,94,7
1,2713,90,3,6479,469,1,8,2012,33,5
2,409,17,2,25472,435,1,12,2001,196,4
3,1150,234,10,23950,529,2,23,2019,79,2
4,2424,390,5,13046,395,2,20,2010,200,4


# Método 1

In [ ]:
# generar una tabla dinámica con lectores en el índice y libros en la columna y valores siendo las calificaciones

pt_df = df.pivot_table(
    columns = 'book_id',
    index = 'reader_id',
    values = 'book_rating'
).fillna(0)

# Convertir a csr matrix
mat = pt_df.values
mat = csr_matrix(mat)

In [ ]:
def normalize(pred_ratings):
  '''
  Esta función normalizará la entrada pred_ratings  
  parámetros:
  pred_ratings (Lista -lista >) : Las calificaciones de predicción 
  '''
  return (pred_ratings - pred_ratings.min()) / (pred_ratings.max() - pred_ratings.min())

In [ ]:
def generate_prediction_df(mat, pt_df, n_factors):
    '''
    Esta función calculará la descomposición de un solo valor de la matriz de entrada
    dado n_factors. Luego generará y normalizará las predicciones de calificación del usuario.
    parámetros:
        mat (CSR Matrix) : matriz scipy csr correspondiente a la tabla dinámica (pt_df)
        pt_df (DataFrame) : pandas dataframe que es una tabla dinámica
        n_factors (Integer) : Número de valores singulares y vectores a calcular. 
                              Debe ser 1 < = n_factors < min(mat.shape). 
    '''
    if not 1 <= n_factors < min(mat.shape):
        raise ValueError("Must be 1 <= n_factors < min(mat.shape)")        
    # factorización matricial
    u, s, v = svds(mat, k = n_factors)
    s = np.diag(s)

    # calcular clasificaciones de pred
    pred_ratings = np.dot(np.dot(u, s), v) 
    pred_ratings = normalize(pred_ratings)
    
    # Convertir a df
    pred_df = pd.DataFrame(
        pred_ratings,
        columns = pt_df.columns,
        index = list(pt_df.index)
    ).transpose()
    return pred_df

In [ ]:
%time pred_df = generate_prediction_df(mat, pt_df, 10)

CPU times: user 1.17 s, sys: 1.79 s, total: 2.97 s
Wall time: 2.21 s


In [ ]:
def recommend_items(pred_df, usr_id, n_recs):
    '''
    Dada una usr_id y pred_df esta función recomendará
    para el usuario.
    
    parámetros:
        pred_df (DataFrame) : generado a partir de la función 'generate_prediction_df'
        usr_id (Integer) : El usuario para el que desea obtener recomendaciones de artículos
        n_recs (Integer) : El número de recomendaciones que desea para este usuario
    '''
    
    usr_pred = pred_df[usr_id].sort_values(ascending = False).reset_index().rename(columns = {usr_id : 'sim'})
    rec_df = usr_pred.sort_values(by = 'sim', ascending = False).head(n_recs)
    return rec_df

In [ ]:
recommend_items(pred_df, 5, 5)

,book_id,sim
0,1108,0.143244
1,1839,0.140945
2,725,0.140516
3,1897,0.140497
4,1341,0.140086


# Método 2

In [ ]:
pt_df = df.pivot_table(
    columns = 'book_id', 
    index = 'author_id', 
    values = 'book_rating'
).fillna(0)

In [ ]:
def find_similar_readers(pt_df, reader_id, n_recs):
    '''
    Esta función encontrará lectores similares al usuario especificado reader_id
    '''
    
    # lector separado de interés y todos los demás lectores
    reader = pt_df[pt_df.index == reader_id]
    other_readers = pt_df[pt_df.index != reader_id]

    # obtener similitud del lector actual y todos los demás lectores
    sim = cosine_similarity(reader, other_readers)[0].tolist()
    idx = other_readers.index.tolist()

    # crear un diccionario de similitud para este usuario w.r.t todos los demás usuarios
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)

    similar_readers = idx_sim[:n_recs]
    readers = [rdr[0] for rdr in similar_readers]

    return readers

In [ ]:
find_similar_readers(pt_df = pt_df, reader_id = 226, n_recs = 5)

[319, 191, 145, 162, 212]

# Método 3

In [ ]:
rdf = df[['reader_id', 'book_id', 'book_rating']]

In [ ]:
#Leer el dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rdf, reader)

In [ ]:
#Dividir el conjunto de datos
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [ ]:
# Utilice user_based true/false para cambiar entre el filtrado colaborativo basado en usuarios o en elementos
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
# ejecutar el modelo entrenado en el conjunto de pruebas
test_pred = algo.test(testset)

In [ ]:
# obtener RMSE
accuracy.rmse(test_pred, verbose=True)

RMSE: 2.9306


2.9306185721359865

In [ ]:
algo.predict(uid = 10, iid = 43)

Prediction(uid=10, iid=43, r_ui=None, est=5, details={'actual_k': 0, 'was_impossible': False})

## Bibliografía:

S.O. (2021e). scipy.sparse.linalg.svds — SciPy v1.7.1 Manual. ScipyOrg. Recuperado 19 de enero de 2022, de https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html

O. (2019, 2 diciembre). Sistemas de recomendación | ¿Qué es el filtrado colaborativo? GraphEverywhere. Recuperado 19 de enero de 2022, de https://www.grapheverywhere.com/sistemas-de-recomendacion-que-es-el-filtrado-colaborativo/

GeeksforGeeks. (2020, 19 agosto). How to Create a Sparse Matrix in Python. Recuperado 19 de enero de 2022, de https://www.geeksforgeeks.org/how-to-create-a-sparse-matrix-in-python/#:%7E:text=Python%E2%80%99s%20SciPy%20gives%20tools%20for%20creating%20sparse%20matrices,sparse%20matrix%20of%20c%20ompressed%20sparse%20column%20format.

PyShark. (2021, 8 diciembre). Cosine Similarity Explained using Python - Machine Learning. Recuperado 19 de enero de 2022, de https://pyshark.com/cosine-similarity-explained-using-python/

Samaniego, T. (2021, 11 febrero). Numpy en Python - Qué es, instalación y sintaxis. Mi Diario Python. Recuperado 19 de enero de 2022, de https://pythondiario.com/2019/05/numpy-en-python.html

Ebrahim, M. (2019, 2 abril). Tutorial De Python Pandas: Iniciando Con DataFrames. Like Geeks. Recuperado 19 de enero de 2022, de https://likegeeks.com/es/tutorial-de-python-pandas/#:%7E:text=Pandas%20es%20una%20biblioteca%20de%20c%C3%B3digo%20abierto%20de,DataFrame%20donde%20almacena%20y%20juega%20con%20los%20datos.






